In [ ]:
import numpy as np
import cv2
import os

In [ ]:
camera_number = 1
frame_count = 0
image_number = 10

#print(os.listdir())

abs_dir_path = "videos/"
VIDEO_NAME = f'out{camera_number}F.mp4'
ROW_POINTS = 5
COLUMNS_POINTS = 7
# Open the video file 
video_capture = cv2.VideoCapture(filename=f'{abs_dir_path}{VIDEO_NAME}')

# Print the number of frames in the video
number_of_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", number_of_frame)

In [ ]:
os.listdir()
#os.chdir('..')

In [ ]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + f'out{camera_number}'
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))


In [ ]:
#findChessboard Flag test
import datetime as dt
import lib.util as util
import lib.Calibration as cal
import cv2 

video_path = '...'
width = 0
height = 0

one = cal.Calibration(video_path, width, height)
skip = 500
img = []
print(f'starting skip of {skip}')
for i in range(skip):
    ret, img = one.video_capture.read()
    if not ret:
        break
print(f'frame after skip')

cv2.namedWindow("test image", cv2.WINDOW_NORMAL) 
  
# Using resizeWindow() 
cv2.resizeWindow("test image", 1920, 1080) 
  
# Displaying the image 
cv2.imshow("test image", img) 

one.video_capture.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

#flags FAST_CHECK -> False FAST_CHECK + NORMALIZE -> False  None -> True FAST_CHECK + NORMALIZE + THRESH 
now = dt.datetime.now()
timestamp = util.getSeconds(now)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
print(f"all flags time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  None)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no flags time: {(t2 - timestamp)}")
print(ret)

##best one!
now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no norm flag time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags= cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"only fast flag time: {(t2 - timestamp)}")
print(ret)


In [3]:
#COMPUTING INTRINSIC PARAMETERS TEST
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm

camera = 'out2F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d',camera)[0]) - 1]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]


img_list = util.loadImagesBatch(input_dir)
img_size =  ((img_list[0]).shape[1], (img_list[0]).shape[0]) #width and height
imgpoints = []
objpoints = []
print(img_size)
objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)

for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray,(CHESS_WIDTH, CHESS_HEIGHT),None, cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0]}')
#no subpixel cases

    #tc std.
tc = (cv2.TERM_CRITERIA_EPS, 0.005)
print(f'tcstd{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}std')
    #tc high accuracy
tc = (cv2.TERM_CRITERIA_EPS, 0.00001)
print(f'tchigh{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}high_acc')
    #tc high accuracy and iteration
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)
print(f'tccombine{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}iter')

#subpixel refinement



(3840, 2160)


100%|██████████| 60/60 [01:09<00:00,  1.15s/it]


first imgpoints: [[[2511.2573 1429.4196]]

 [[2598.417  1436.1726]]

 [[2683.5823 1443.8806]]

 [[2770.335  1450.5688]]

 [[2856.677  1455.0978]]

 [[2508.044  1511.5557]]

 [[2592.4006 1518.7101]]

 [[2677.578  1525.2919]]

 [[2761.9016 1533.4159]]

 [[2847.2607 1539.0739]]

 [[2503.0168 1589.766 ]]

 [[2587.337  1596.8707]]

 [[2669.858  1603.93  ]]

 [[2753.4512 1611.6711]]

 [[2836.2014 1620.0504]]

 [[2497.8682 1665.489 ]]

 [[2580.1475 1672.1979]]

 [[2661.5596 1680.0149]]

 [[2743.9783 1687.0309]]

 [[2826.105  1694.951 ]]

 [[2492.052  1738.472 ]]

 [[2572.7935 1745.3141]]

 [[2653.5513 1752.1088]]

 [[2733.3428 1760.4625]]

 [[2814.617  1767.7542]]

 [[2486.004  1807.653 ]]

 [[2565.0032 1815.362 ]]

 [[2644.5217 1822.934 ]]

 [[2723.3694 1829.8905]]

 [[2802.4587 1837.0004]]

 [[2479.1455 1873.6735]]

 [[2557.095  1881.2943]]

 [[2635.3777 1889.1616]]

 [[2711.7727 1897.1212]]

 [[2790.2825 1903.6965]]]
tcstd(2, 0.005)
tchigh(2, 1e-05)
tccombine(3, 0.005, 50)


True

In [1]:
#UNDISTORTION TEST
import numpy as np
import cv2
import os
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal

camera = 'out2F'
image = 'chessboard55'

und_dir = f'undistorted/{camera}'
os.makedirs(und_dir, exist_ok=True)

# Remove all files in the directory
for file in os.listdir(und_dir):
    os.remove(os.path.join(und_dir, file))

filepath = f'samples/{camera}/{image}.jpg'
img = cv2.imread(filename=filepath)

#showing original image
util.showImage(img, 'original')

#refining camera matrix
path = '2Fcorners.json'
dict = util.LoadJSON(path)
h, w = img.shape[:2]
#standard values, taken from the json
std_mtx = np.array(dict['mtx'])
std_dist = np.array(dict['dist'])
print(f"numpy matrix: {std_mtx}")
new_mtx, roi = cv2.getOptimalNewCameraMatrix(std_mtx, std_dist, (w,h), 1, (w,h))

x, y, w, h = roi

#undistort the image, corrected with the new computed camera matrix
new_dist = np.array([])
und_img = cv2.undistort(src=img, cameraMatrix= std_mtx, dst= new_dist ,distCoeffs= std_dist, newCameraMatrix= new_mtx)
util.showImage(und_img, 'undistorted - new camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx_no.jpg',und_img)
#print((np.size(np.array(und_img),0),np.size(np.array(und_img),1)))
# crop
crop_und_img = und_img[y:y+h, x:x+w]
util.showImage(crop_und_img, 'undistorted - new camera matrix - cropped')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx.jpg', crop_und_img)
#no new camera matrix
und_img = cv2.undistort(src=img, cameraMatrix= std_mtx, dst= new_dist ,distCoeffs= std_dist)
util.showImage(und_img, 'undistorted - standard camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_std_mtx.jpg',und_img)
# crop 
# no_crop_und_img = und_img[y:y+h, x:x+w]
#util.showImage(und_img, 'undistorted - standard camera matrix - cropped')


numpy matrix: [[2.04424120e+03 0.00000000e+00 1.96990101e+03]
 [0.00000000e+00 2.04989774e+03 1.16571829e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


True

In [2]:
#image ordered calibration tests 
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm

camera = 'out2F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d',camera)[0]) - 1]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]



img_list = util.loadImagesBatch(input_dir)
img_list_sorted = util.loadImagesBatch(input_dir, True)

img_size =  ((img_list[0]).shape[1], (img_list[0]).shape[0]) #width and height
imgpoints = []
objpoints = []
print(img_size)
objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)


for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray,(CHESS_WIDTH, CHESS_HEIGHT),None, cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        #print('found')
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0][0]}')
print(len(imgpoints))
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}std_fetching')

imgpoints = []
objpoints = []

for img in tqdm(img_list_sorted):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray,(CHESS_WIDTH, CHESS_HEIGHT),None, cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0][0]}')
print(len(imgpoints))
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
ordered_json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(ordered_json_camera_matrix, f'{camera}sorted_fetching')




list: ['chessboard17.jpg', 'chessboard37.jpg', 'chessboard46.jpg', 'chessboard26.jpg', 'chessboard25.jpg', 'chessboard53.jpg', 'chessboard34.jpg', 'chessboard52.jpg', 'chessboard43.jpg', 'chessboard39.jpg', 'chessboard8.jpg', 'chessboard0.jpg', 'chessboard32.jpg', 'chessboard50.jpg', 'chessboard30.jpg', 'chessboard44.jpg', 'chessboard15.jpg', 'chessboard10.jpg', 'chessboard5.jpg', 'chessboard59.jpg', 'chessboard47.jpg', 'chessboard31.jpg', 'chessboard40.jpg', 'chessboard42.jpg', 'chessboard29.jpg', 'chessboard54.jpg', 'chessboard33.jpg', 'chessboard16.jpg', 'chessboard41.jpg', 'chessboard7.jpg', 'chessboard24.jpg', 'chessboard57.jpg', 'chessboard2.jpg', 'chessboard21.jpg', 'chessboard1.jpg', 'chessboard56.jpg', 'chessboard38.jpg', 'chessboard13.jpg', 'chessboard11.jpg', 'chessboard18.jpg', 'chessboard22.jpg', 'chessboard48.jpg', 'chessboard4.jpg', 'chessboard3.jpg', 'chessboard9.jpg', 'chessboard58.jpg', 'chessboard27.jpg', 'chessboard6.jpg', 'chessboard20.jpg', 'chessboard45.jpg', 'ch

100%|██████████| 60/60 [00:55<00:00,  1.08it/s]


first imgpoints: [[[2511.2573 1429.4196]]

 [[2598.417  1436.1726]]

 [[2683.5823 1443.8806]]

 [[2770.335  1450.5688]]

 [[2856.677  1455.0978]]

 [[2508.044  1511.5557]]

 [[2592.4006 1518.7101]]

 [[2677.578  1525.2919]]

 [[2761.9016 1533.4159]]

 [[2847.2607 1539.0739]]

 [[2503.0168 1589.766 ]]

 [[2587.337  1596.8707]]

 [[2669.858  1603.93  ]]

 [[2753.4512 1611.6711]]

 [[2836.2014 1620.0504]]

 [[2497.8682 1665.489 ]]

 [[2580.1475 1672.1979]]

 [[2661.5596 1680.0149]]

 [[2743.9783 1687.0309]]

 [[2826.105  1694.951 ]]

 [[2492.052  1738.472 ]]

 [[2572.7935 1745.3141]]

 [[2653.5513 1752.1088]]

 [[2733.3428 1760.4625]]

 [[2814.617  1767.7542]]

 [[2486.004  1807.653 ]]

 [[2565.0032 1815.362 ]]

 [[2644.5217 1822.934 ]]

 [[2723.3694 1829.8905]]

 [[2802.4587 1837.0004]]

 [[2479.1455 1873.6735]]

 [[2557.095  1881.2943]]

 [[2635.3777 1889.1616]]

 [[2711.7727 1897.1212]]

 [[2790.2825 1903.6965]]]
60


100%|██████████| 60/60 [00:55<00:00,  1.09it/s]


first imgpoints: [[1669.6437 1672.8748]]
60


True